# IN4080: obligatory assignment 1 (Autumn 2025)
 
Mandatory assignment 1 consists of three parts. In Part 1 (6 points), you will test and improve on a BPE (Byte-Pair-Encoding) tokenizer . In Part 2 (7 points), you will estimate an N-gram language model, based on a training corpus and the tokenizer you worked on in Part 1. Finally, in Part 3 (7 points), you will develop a basic classification model to distinguish between Bokmål and Nynorsk sentences.

You should answer all three parts. You are required to get at least 12/20 points to pass. The most important is that you try to answer each question (possibly with some mistakes), to help you gain a better and more concrete understanding of the topics covered during the lectures. There are also bonus questions for those of you who would like to deepen their understanding of the topics covered by this assignment.

- We assume that you have read and are familiar with IFI’s requirements and guidelines for mandatory assignments, see [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-mandatory.html) and [here](https://www.uio.no/english/studies/examinations/compulsory-activities/mn-ifi-guidelines.html).
- This is an individual assignment. You should not deliver joint submissions. 
- You may redeliver in Devilry before the deadline (__Friday, September 12 at 23:59__), but include all files in the last delivery.
- Only the last delivery will be read! If you deliver more than one file, put them into a zip-archive. You don't have to include in your delivery the files already provided for this assignment. 
- Name your submission _your\_username\_in4080\_mandatory1_
- You can work on this assignment either on the IFI machines or on your own computer. 

*The preferred format for the assignment is a completed version of this Jupyter notebook*, containing both your code and explanations about the steps you followed. We want to stress that simply submitting code is __not__ by itself sufficient to complete the assignment - we expect the notebook to also contain explanations of what you have implemented, along with motivations for the choices you made along the way. Preferably use whole sentences, and mathematical formulas if necessary. Explaining in your own words (using concepts we have covered through in the lectures) what you have done and reflecting on your solution is an important part of the learning process - take it seriously!

Regarding the use of LLMs (ChatGPT or similar): you are allowed to use them as 'sparring partner', for instance to clarify something you have not understood. However, you are __not__ allowed to use them to generate solutions (either in part or in full) to the assignment tasks. 

__Technical tip__: Some of the tasks in this assignment will require you to extend methods in classes that are already partly implemented. To implement those methods directly in a Jupyter notebook, you can use the function `setattr` to attach a method to a given class: 

```python
class A:
    pass
a = A()

def foo(self):
    print('hello world!')
    
setattr(A, 'foo', foo)
```

First, make sure that all required modules are installed:

In [1]:
%pip install tqdm numpy scikit_learn

Note: you may need to restart the kernel to use updated packages.


## Part 1 : Tokenisation

We will start by building a basic tokenizer relying on white space and punctuation. 

__Task 1.1__ (2 points): Implement the method `split` below such that it takes a text as input and outputs a list of tokens. The tokenisation should simply be done by splitting on white space, except for punctuation markers and other symbols (`.,:;!?-()"`), which should correspond to their own token. For instance, the sentence "Pierre, who works at NR, also teaches at UiO." should be split into 12 tokens.

In [1]:
from typing import List
import re

def basic_tokenize(text: str) -> List[str]:
    """The method should split the text on white space, except for punctuation
    markers that should be considered as tokens of their own (even in the 
    absence of white space before or after their occurrence)"""

    # Implement here your basic tokenisation
    
    return re.findall(r"\w+|[.,:;!?\-()\"]",text)

In [2]:
a = "Pierre, who works at NR, also teaches at UiO."
token_a = basic_tokenize(a)
len(token_a)

12

Shu_explanation:
Since the regular expression package re is imported, we can use the findall function to extract tokens. The pattern \w+ is used to capture words. Still, we need to handle punctuation symbols so that they become separate tokens. 

Obs!: - and " must be handled with a backslash (\-), otherwise we will get a “bad character” error.

We will now run the tokeniser on a small corpus, the [Norwegian Dependency Treebank](https://www.nb.no/sprakbanken/en/resource-catalogue/oai-nb-no-sbr-10/) (the corpus has been annotated with morphological features, syntactic functions and hierarchical structures, but we'll simply use here the raw text and discard all the annotation layers). We provide you with the data in the files `ndt_train_lm.txt` and `ndt_test_lm.txt`. 

__Task 1.2__ (1 point): Run the tokenizer you have implemented on `ndt_test_lm.txt`. How many tokens were extracted? And how many types (distinct words) were there? 

In [85]:
%pip install transformers tokenizers

Note: you may need to restart the kernel to use updated packages.


In [2]:
with open('ndt_train_lm.txt', 'r', encoding='utf-8-sig') as f:
  text_train = f.read()

In [4]:
len(test_tokens)

258079

Shu_solution: the output of basic_tokenize function is a list of tokens. The difference between tokens and types(distinct words) is that types do not have repeated words. So we just need to take away the repeated words in a list. We can simply use set() to do that.

In [5]:
test_types = set(test_tokens)
len(test_types)

30006

We shall now use Byte-Pair Encoding (BPE) to limit the vocabulary of the tokenizer to 5,000.  An initial implementation of the algorithm is provided below.

In [6]:
from typing import Dict, List, Tuple, Iterator
import numpy as np
from tqdm.notebook import tqdm

class BPETokenizer:
    """Tokenizer based on the Byte-Pair Encoding algorithm. 
    Note: the current implementation is limited to Latin characters (ISO-8859-1)"""
    #def __init__(self, train_corpus_file: str, vocab_size = 500):
    def __init__(self, train_corpus_file: str, vocab_size = 5000): 
        """Creates a new BPE tokenizer, with merge pairs found using the given
        corpus file. The extraction of merge pairs stops when a vocabulary of 
        size vocab_size is reached."""

        # List of string pairs that should be merged when tokenizing
        # Example: ('e', 't'), which means that 'et' is a possible subword
        # Each string pair is mapped to an unique index number
        # (corresponding to their position in the self.vocab list)
        self.merge_pairs = {}

        # We add as basic vocab all characters of the extended ASCII
        self.vocab = [chr(i) for i in range(256)]

        ## Shu add###
        # identify punct
        self.punct = set('.,:;!?-()"')
        #########
       

        with open(train_corpus_file) as fd:

            # We first read the corpus, split on white space, and counts the
            # occurrences of each distinct word
            #print("Counting word occurrences in corpus %s"%train_corpus_file, end="...", flush=True)
            text = fd.read()
            vocabulary_counts = {}
            for token in text.split():
                vocabulary_counts[token] = vocabulary_counts.get(token, 0) + 1
            #print("Done")

            # We then iteratively extend the list of merge pairs until we
            # reach the desired size. Note: to speed up the algorithm, we 
            # extract n merge pairs at each iteration
            #progress_bar = tqdm(total=vocab_size)
            while len(self.vocab) < vocab_size:
                most_common_pairs = self.get_most_common_pairs(vocabulary_counts)      
                for common_pair in most_common_pairs:
                    self.merge_pairs[common_pair] = len(self.vocab)
                    self.vocab.append("".join(common_pair))
                    if len(self.vocab) >= vocab_size:
                        break
                #progress_bar.update(len(most_common_pairs))
                #print("Examples of new subwords:", ["".join(pair) for pair in most_common_pairs][:10]) ##
            
    def get_most_common_pairs(self, vocabulary_counts: Dict[str,int], 
                              n:int=200) -> List[Tuple[str,str]]:
        """Given a set of distinct words along with their corresponding number 
        of occurrences in the corpus, returns the n most frequent pairs of subwords.       
        """

        # We count the frequencies of consecutive subwords in the vocabulary list
        pair_freqs = {}
        for word, word_count in vocabulary_counts.items():
            subwords = self.tokenize_word(word)
            
            for i in range(len(subwords)-1):
                byte_pair = (subwords[i], subwords[i+1])
                
                ##Shu add: skip punctuation-involved pairs########
                if any(char in self.punct for char in byte_pair[0]) or any(char in self.punct for char in byte_pair[1]):
                    continue
                ################################################    
        
                pair_freqs[byte_pair] = pair_freqs.get(byte_pair, 0) + word_count

        # And return the most frequent ones
        most_freq_pairs = sorted(pair_freqs.keys(), key=lambda x: pair_freqs[x])[::-1][:n]
        return most_freq_pairs

    def __call__(self, input:str, show_progress_bar=True) -> Iterator[str]:
        """Tokenizes a full text"""

        # We first split into whitespace-separated tokens, and then in subwords
        words = input.split()
        #for word in tqdm(words) if show_progress_bar else words:
        for word in words:
            subwords = self.tokenize_word(word)
            for subword in subwords:
                yield subword
                

    def tokenize_word(self, word):
        """Splits the word into subwords, according to the merge pairs 
        currently stored in self.merge_pairs."""

        # We start with a list of characters
        # (+ a final character to denote the end of the word)    
        splits = list(word) + [" "]

        # We continue until there is nothing left to be merged
        while len(splits)>=2:

            # We extract consecutive subword pairs
            pairs = [(splits[i], splits[i+1]) for i in range(len(splits)-1)]

            # We find the "best" pair of subwords to merge -- that is, the one with the 
            # lowest position in the list of merge rules
            best_pair_to_merge = min(pairs, key=lambda x: self.merge_pairs.get(x, np.inf))
            if best_pair_to_merge in self.merge_pairs:

                # We then merge the two subwords
                for i in range(len(splits)-1):
                    if (splits[i], splits[i+1]) == best_pair_to_merge:
                        merged_subword = self.vocab[self.merge_pairs[best_pair_to_merge]]
                        splits = splits[:i] + [merged_subword] + splits[i+2:]
                        break
            else:
                break
        return splits

__Task 1.3__ (1 point): Learn the BPE tokenizer on the `ndt_train_lm.txt` corpus, and then apply this tokenizer on `ndt_test_lm.txt`. Print the number of tokens and types (distinct subwords) obtained by this tokenizer on the test data. How do those numbers compare to the ones obtained with the basic tokenizer you had implemented earlier ? 

In [7]:
#Train the BPETokenizer based on training data.
tokenizer = BPETokenizer('ndt_train_lm.txt',vocab_size = 5000)

In [8]:
#apply the trained tokenizer to the test data
test_new_tokens = []
for token in tokenizer(text_test):
    if token != " ": #this is because in the BPE tokenizer, we add a " "(space) to denote the end of the word. We need to get rid of it. 
        test_new_tokens.append(token)
#test_new_tokens

In [9]:
len(test_new_tokens)

398709

In [10]:
len(set(test_new_tokens))

4341

Shu_explain:
We can see that the token length using the BPETokenizer has increased compared to when using the basic tokenizer. The reason is in basic tokenizer the minimum unit is word. However, the minimum unit in BPETokenizer is subword. For example, the token 'Lam' has turned to 'La','m'. This explains why the number of tokens using BPETokenizer increased compared to the basic tokenizer.

The number of types has been greatly reduces when using BPETokenizer. The reason is since word has been splitted to subword, the subword can be combined in different orders to form different word. So the vocabulary of subwords is more compact and flexible. Another reason is, BPETokenizer has limit the vocabulary to a subword up to 5,000. So the BPETokenizer has filtered out the rare tokens compared to basic tokenizer. Thus BPETokenizer produces a much more concise and manageable vocabulary.

__Task 1.4__ (2 points): The current BPE implementation is that it treats all characters in the same manner. A rather inconvenient side effect is that letters may be merged together with punctuation markers (like 'ing', ',' --> 'ing,'), if they are not separated by white space. Modify the implementation of the BPE algorithm above to prevent punctuation markers to be merged with letters. 

Shu_solution: 
In this task, we need to avoid subword merging with punctuation. So first, we define a set of punctuation characters as self.punct.
Then before merging two common_pairs, we need to identify if punctuation exist in either common_pair. If yes, we skip merging using continue. otherwise, we proceed the combine work. 

__Task 1.5__ (_optional, 2 extra points_): In a [tweet](https://x.com/karpathy/status/1759996551378940395) published last year, the well-known AI researcher Andrej Karpathy stressed that many of the current limitations of Large Language Models are actually a product of the tokenisation step. Explain at least 4 of the problems he mentioned in his tweet (you can of course search online, or watch Karpathy's own video lecture on tokenization).

## Part 2: N-gram language models

We will now train simple N-gram language models on the NDT corpus, using the tokenizers we have developed in Part 1.

Here is the skeleton of the code:

In [12]:
import numpy as np
from abc import abstractmethod

class LanguageModel:
    """Generic class for running operations on language models, using a BPE tokenizer"""

    def __init__(self, tokenizer: BPETokenizer):
        """Build an abstract language model using the provided tokenizer"""
        self.tokenizer = tokenizer
 
    @abstractmethod
    def predict(self, context_tokens: List[str]):
        """Given a list of context tokens (=previous tokens), returns a dictionary
          mapping each possible token to its probability"""
        raise NotImplementedError()
    
    @abstractmethod
    def get_perplexity(self, text: str):
        """Computes the perplexity of the given text according to the LM"""

        print("Tokenising input text:")
        tokens = list(self.tokenizer(text))
        
        print("Computing perplexity:")
        log_probs = 0
        #for i in tqdm(range(len(tokens))):
        for i in range(len(tokens)):    
            context_tokens = ["<s>"] + tokens[:i]
            predict_distrib = self.predict(context_tokens)

            # We add the log-probabilities
            log_probs += np.log(predict_distrib[tokens[i]])
            
        perplexity = np.exp(-log_probs/len(tokens))
        return perplexity

class NGramLanguageModel(LanguageModel):
    """Representation of a N-gram-based language model"""

    def __init__(self, training_corpus_file: str, tokenizer:BPETokenizer, ngram_size:int=3,
                  alpha_smoothing:float=1):
        """Initialize the N-gram model with:
        - a file path to a training corpus to estimate the N-gram probabilities
        - an already learned BPE tokenizer
        - an N-gram size
        - a smoothing parameter (Laplace smoothing)"""
        
        LanguageModel.__init__(self, tokenizer)
        self.ngram_size = ngram_size
        
        # We define a simple backoff distribution (here just a uniform distribution)
        self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

        # Dictionary mapping a context (for instance the two preceding words if ngram_size=3)
        # to another dictionary specifying the probability of each possible word in the 
        # vocabulary. The context should be a tuple of tokens.
        self.ngram_probs = {}
        with open(training_corpus_file) as fd:   

            # based on the training corpus, tokenizer, ngram-size and smoothing parameter,
            # fill the self.ngram_probs with the correct N-gram probabilities  
            raise NotImplementedError()
 

    def predict(self, context_tokens: List[str]):
        """Given a list of preceding tokens, returns the probability distribution 
        over the next possible token."""

        # We restrict the contextual tokens to (N-1) tokens
        context_tokens = tuple(context_tokens[-self.ngram_size+1:])

        # If the contextual tokens were indeed observed in the corpus, simply
        # returns the precomputed probabilities
        if context_tokens in self.ngram_probs:
            return self.ngram_probs[context_tokens]
        
        # Otherwise, we return a uniform distribution over possible tokens
        else:
            return self.default_distrib

__Task 2.1__ (6 points): Complete the initialization method `__init__` to estimate the correct N-gram probabilities (with smoothing) based on the corpus. Don't worry about making your implementation super-efficient (although you can if you wish).

In [112]:
from collections import defaultdict, Counter

def __init__(self, training_corpus_file: str, tokenizer:BPETokenizer, ngram_size:int=2, alpha_smoothing=0.1):
        """Initialize the N-gram model with:
        - a file path to a training corpus to estimate the N-gram probabilities
        - an already learned BPE tokenizer
        - an N-gram size
        - a smoothing parameter (Laplace smoothing)"""

        

    
        LanguageModel.__init__(self, tokenizer)
        self.ngram_size = ngram_size
        
        # We define a simple backoff distribution (here just a uniform distribution)
        self.default_distrib = {token:1/len(tokenizer.vocab) for token in tokenizer.vocab}

        # Dictionary mapping a context (for instance the two preceding words if ngram_size=3)
        # to another dictionary specifying the probability of each possible word in the 
        # vocabulary. The context should be a tuple of tokens.
        self.ngram_probs = {}
    
        with open(training_corpus_file) as fd:  
            
            # ADD HERE YOUR CODE TO FILL THE VALUES IN self.ngram_probs

            #generate tokens
            text = fd.read()
            #tokens = list(tokenizer(text))
            
            tokens = ["<s>"] * (self.ngram_size - 1) + list(tokenizer(text))
            #print("Number of tokens in training data:", len(tokens))
            #print("First 20 tokens:", tokens[:20])


            # construct the right data format according to the ngram_size, when ngram_size=3
            # it means ngrams is a list with tuples (each tuple size=3)
            # I construct it to list type, because it is easier to use Counter in list
            # The context is basically ngram drop the last element,which is the target
            
            ngrams = [tuple(tokens[i:i+self.ngram_size]) for i in range(len(tokens)-self.ngram_size+1)]
            #ngrams = [tuple(tokens[i:i+self.ngram_size]) for i in range(100-self.ngram_size+1)]
            contexts = [item[:-1] for item in ngrams]

            ngram_counts = Counter(ngrams)
            context_counts = Counter(contexts)
            #print(ngram_counts,'--------------',context_counts)
            print(f"ngram_counts: {len(ngram_counts)}, context_counts: {len(context_counts)}")


            ## Now calculate the probability based on the count result
            ## Here we apply Markov assumption, which simplifies the problem by restricting the conditional probabilities
            ## to the k previous words. when ngram_size=3, k=2
            ## Here we need to consider Laplace smoothing to avoid target appears in testing set not training set
            ## We limit the computation by only run a fraction of context_counts (20000 context tokens)


            for context_tokens in list(context_counts)[:20000]: #[改] Here we run only a fraction of context_counts
                denominator = context_counts[context_tokens] + alpha_smoothing * (len(self.tokenizer.vocab))
                distrib = {
                            token: (ngram_counts.get((*context_tokens, token), 0) + alpha_smoothing) / denominator
                            for token in self.tokenizer.vocab
                            #for token in self.tokenizer.vocab[:100]
                        }
                self.ngram_probs[context_tokens] = distrib
                
            ctx = ('<s>','No',)
            sorted_probs = sorted(self.ngram_probs[ctx].items(), key=lambda x: x[1], reverse=True)
            print("Top 5 predictions:", sorted_probs[:5])                       

setattr(NGramLanguageModel, '__init__', __init__)

Shu_explaination:
In this task, we first tokenize the ndt_train_lm and generate the tokens for the training data, using the BPETokenizer we implemented in task 1. 
Then we need to construct the data to the right format, we did a little test sample below to illustrate the format we want to have in the ngrams, when ngram_size=3, we want to have 3 tokens in 1 tuple, in this tuple, the last element is our target, and first two are our context. 
When we have ngrams, it is easy to get contexts, simple drop the last elements of ngram then we have context.
Now we have the right format data ngrams and contexts. We can use Counter function to count how many times each tuple occurs. 
1. ngram_counts: to count the number of occurance of (context1, context2, target)
2. context_counts: to count the occurrance of the (context1, context2)
When we have all the count numbers, we can calculate probabilities using Markov assumption.
We use for loop of tokens, to add token to the context and check if it appears in ngrams_counts, if not apear, then value = 0
When target doesn't exist in training set,then the predict probability will be 0, and in the perplexity we use log, then it is meaningless.So we need to introduce Laplace smoothing. By doing this, even if the target never appear in the training set, it will still have a very small probability. In LLM next word prediction, we only choose the token with the hightest probability.
After probabilitiy is calculated, put the context_tokens and probability into ngram_probs.

Shu_reflection:
When constructing the ngram_probs dictionary, we rely on nested loops. This causes the data size to grow extremely large — roughly proportional to context size × vocabulary size. As a result, building the full distribution can be very time-consuming. During testing, we reduced both the vocabulary size and the number of contexts, which allowed us to quickly verify the correctness of our implementation. However, this simplification introduces certain limitations, since it does not reflect the performance of the full model. We will revisit this issue in the next task.

In [14]:
# test
from collections import defaultdict, Counter
a = [1,2,3,4,1,2,4,5,6]
b = [tuple(a[i:i+3]) for i in range(len(a)-3+1)]
c= [item[:-1] for item in b]
print(b,Counter(b),c,Counter(c))
#ngrams = [tuple(tokens[i:i+self.ngram_size]) for i in range(len(tokens)-self.ngram_size+1)]
#contexts = [ng[:-1] for ng in ngrams]

[(1, 2, 3), (2, 3, 4), (3, 4, 1), (4, 1, 2), (1, 2, 4), (2, 4, 5), (4, 5, 6)] Counter({(1, 2, 3): 1, (2, 3, 4): 1, (3, 4, 1): 1, (4, 1, 2): 1, (1, 2, 4): 1, (2, 4, 5): 1, (4, 5, 6): 1}) [(1, 2), (2, 3), (3, 4), (4, 1), (1, 2), (2, 4), (4, 5)] Counter({(1, 2): 2, (2, 3): 1, (3, 4): 1, (4, 1): 1, (2, 4): 1, (4, 5): 1})


__Task 2.2__ (1 point): Train your language model in `ndt_train_lm.txt`, and compute its perplexity on the test data in `ndt_test_lm.txt`. The perplexity can be computed by calling the method `get_perplexity`. <br>
(_Note_: if the training takes too much time, feel free to stop the process after looking at a fraction of the corpus, at least while you are testing/developing your training setup).

In [15]:
lm = NGramLanguageModel("ndt_train_lm.txt",tokenizer, ngram_size=3, alpha_smoothing=0.1)

  0%|          | 0/872824 [00:00<?, ?it/s]

ngram_counts: 624814, context_counts: 262225


  0%|          | 0/20000 [00:00<?, ?it/s]

Top 5 predictions: [('kre ', 0.00219560878243513), ('\x00', 0.00019960079840319363), ('\x01', 0.00019960079840319363), ('\x02', 0.00019960079840319363), ('\x03', 0.00019960079840319363)]


In [25]:
with open('./ndt_test_lm.txt', 'r', encoding='utf-8-sig') as f:
    print("text_test size: {len(f.read())}")
    text_test=f.read()
    #text_test=f.read(50000)

In [26]:
test_perplexity = lm.get_perplexity(text_test)
print(test_perplexity)

Tokenising input text:


  0%|          | 0/226096 [00:00<?, ?it/s]

Computing perplexity:


  0%|          | 0/428326 [00:00<?, ?it/s]

1488.310784102801


Shu_reflection:
As mentioned in the previous task, I limited the size of context_counts to 20,000 (compared to the original 262,225). This reduction achieved computational efficiency but resulted in insufficient training and consequently a higher perplexity. When I increased the size of context_counts, there is a clear improvement in perplexity. Similarly, for efficiency in testing, I restricted the test data size to 50000 tokens. This makes the computation of perplexity much quickly. When I later increased the test data to its full size, the perplexity only improved slightly. One key learning is that in real scenarios, it may be sufficient to evaluate perplexity on a reasonable proportion of the test data. This approach is significantly time-saving and energy-efficient, while still providing reliable results.

__Task 2.3__ (_optional_, 4 bonus points): Improve the language model you have just developed. You can choose to focus on improving your model through a backoff mechanism, interpolation, or both. Once you are done, compute the perplexity again on the test corpus to ensure the language model has indeed improved.

## Part 3: Text classification

We will finally use the texts from the Norwegian Dependency Treebank for a classification task -- more precisely to determine whether a sentence is likely to be written in Bokmål or Nynorsk. To this end, we will use a simple bag-of-word setup (or more precisely a bag-of-_subwords_, since we will rely on the subwords extracted using BPE) along with a logistic regression model. As there is only two, mutually exclusive classes, you can view the task as a binary classification problem. 

The training data is found in `ndt_train_class.txt` and simply consists of a list of sentences, each sentence being mapped to a language form (Bokmål: `nob` or Nynorsk: `nno`). The language form is written at the end of each line, separated by a `\t`. Note the training examples are currently _not_ shuffled.

To train and apply your classifier, the easiest is to use the [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) model from `scikit-learn`.

__Task 3.1__ (2 points): Create a `N x V` matrix in which each line corresponds to a training example (out of `N` training instances) and each row corresponds to an individual feature, in this case the presence/absence of a particular subword in the sentence. In other words, there should be a total of `V` features, where `V` is the total size of the vocabulary for our BPE tokenizer. Also create a vector of length `N` with a value of `1` if the sentence was marked as Nynorsk, and 0 if is was marked as Bokmål. 

In [11]:
import pandas as pd
with open('ndt_train_class.txt', 'r', encoding='utf-8-sig') as f:
  text_train = f.read()
b = [sent.split('\t') for sent in text_train.split('\n')]
df =  pd.DataFrame(data=b,columns=['instances','label'])
df

,instances,label
0,Jeg husker også Jens Stoltenbergs uttalelse ti...,nob
1,"Jeg har fulgt med i mange år, og ikke sett noe...",nob
2,Folk i Albania er stolte over NATO-medlemskapet.,nob
3,Spesielt i Lindesnes og Lyngdal er det svært m...,nob
4,"""Om ikkje det er ein mannleg ønskedraum, så ve...",nno
...,...,...
29901,Libero er ikke alene om å pushe ensidig baby- ...,nob
29902,"Legg appelsinsteinane i eit klede, og knyt att...",nno
29903,Dette er boka om Jerusalem som eg alltid har h...,nno
29904,Martin Bengtsson spiller denne siste sangen fo...,nob


In [12]:
df["label"] = df["label"].map({"nob": 0, "nno": 1})
df

,instances,label
0,Jeg husker også Jens Stoltenbergs uttalelse ti...,0.0
1,"Jeg har fulgt med i mange år, og ikke sett noe...",0.0
2,Folk i Albania er stolte over NATO-medlemskapet.,0.0
3,Spesielt i Lindesnes og Lyngdal er det svært m...,0.0
4,"""Om ikkje det er ein mannleg ønskedraum, så ve...",1.0
...,...,...
29901,Libero er ikke alene om å pushe ensidig baby- ...,0.0
29902,"Legg appelsinsteinane i eit klede, og knyt att...",1.0
29903,Dette er boka om Jerusalem som eg alltid har h...,1.0
29904,Martin Bengtsson spiller denne siste sangen fo...,0.0


In [13]:
df["label"].isnull().sum()

1

In [14]:
df.dropna(inplace=True)
df

,instances,label
0,Jeg husker også Jens Stoltenbergs uttalelse ti...,0.0
1,"Jeg har fulgt med i mange år, og ikke sett noe...",0.0
2,Folk i Albania er stolte over NATO-medlemskapet.,0.0
3,Spesielt i Lindesnes og Lyngdal er det svært m...,0.0
4,"""Om ikkje det er ein mannleg ønskedraum, så ve...",1.0
...,...,...
29900,Broren John har selv vært rammet av kontroverser.,0.0
29901,Libero er ikke alene om å pushe ensidig baby- ...,0.0
29902,"Legg appelsinsteinane i eit klede, og knyt att...",1.0
29903,Dette er boka om Jerusalem som eg alltid har h...,1.0


In [15]:
text = " ".join(df["instances"].tolist())
with open("ndt_train_class_text.txt","w", encoding="utf-8") as f:
    f.write(text)

Shu_explaination:
When tokenize our training data, I limit the vocab_size to 800 to reduce the computation of bag-of-words feature vector N(29905)*vocab_size. We expect the discriminator would be 'ikke' and 'ikkje'. When the vocab_size is downsized to 800, we can see these discriminator still exist in the subwords. 
Shu_reflection:
The computation is huge and I have to deactivate the tqdm and the progress_bar to avoid Error:IOPub message rate exceeded.

In [16]:
#Train the BPETokenizer based on training data.
tokenizer_t3 = BPETokenizer('ndt_train_class_text.txt',vocab_size = 800)

Shu_explaination:
In Scikit-Learn, we can use CountVectorizer or TfIdVectorizer to form the Bag-of-words feature vectors. We chose TfIdVectorizer it can handle discriminator much better. It down-weight the very frequent subword and up-weight the less frequent word but more discriminative subwords. 

In [18]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [55]:
#vectorizer = TfidfVectorizer(analyzer=tokenizer_t3, lowercase=False) 
vectorizer = TfidfVectorizer(analyzer=tokenizer_t3,lowercase=False,
                             vocabulary={tok: i for i, tok in enumerate(tokenizer_t3.vocab)})
X_train = vectorizer.fit_transform(df["instances"])

In [56]:
y_train = df['label']

__Task 3.2__ (2 points): Use the data matrix you have just filled to train a logistic regression model (see the documentation on [`LogisticRegression`](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html) for more details). We recommend to use the `liblinear` solver. 

In [57]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression(solver="liblinear", random_state=0)
clf.fit(X_train, y_train)

,penalty,'l2'
,dual,False
,tol,0.0001
,C,1.0
,fit_intercept,True
,intercept_scaling,1
,class_weight,None
,random_state,0
,solver,'liblinear'
,max_iter,100
,multi_class,'deprecated'


__Task 3.3__ (1 point): Now apply the learned logistic regression model to the test set in `ndt_test_class.txt`, and evaluate its performance in terms of accuracy, recall and precision (you can use the functionalities in `sklearn.metrics` to compute those easily).

In [58]:
# import test data, and preprocess them, splitting instances and label, map label to binary, drop null data
with open('ndt_test_class.txt', 'r', encoding='utf-8-sig') as f:
  text_test = f.read()
c = [sent.split('\t') for sent in text_test.split('\n')]
df_test =  pd.DataFrame(data=c,columns=['instances','label'])
df_test["label"] = df_test["label"].map({"nob": 0, "nno": 1})
df_test.dropna(inplace=True)

In [59]:
# apply trained vectorizer on instances to generate the bag-of-words feature vector:
X_test = vectorizer.transform(df_test["instances"])
y_pred = clf.predict(X_test)

In [60]:
y_pred

array([1., 0., 0., ..., 1., 1., 0.])

In [61]:
from sklearn.metrics import accuracy_score, precision_score, recall_score
y_test = df_test['label']

accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test,y_pred)

print(f'accuracy: {accuracy}, precision: {precision}, recall: {recall}')

accuracy: 0.8983797796500324, precision: 0.916300848697688, recall: 0.862772113529898


__Task 3.4__ (2 points): Inspect the weights learned by your logistic regression model (in `coef_`) and find the 5 subwords that contribute _the most_ to the classification of the sentence in Nynorsk. Also find the 5 subwords that contribute the most to the classification of the sentence in Bokmål. Do those weights make sense, according to what you know about Bokmål and Nynorsk ? 

In [53]:
#check weights learned by logistic regression model.
clf.coef_.shape

(1, 666)

In [54]:
len(tokenizer_t3.vocab)

800

Shu_explaination:
We can see that the weight shape contains only 666 columns, but our tokenizer vocabulary we set to 800. The reason is when we used TfIdVectorizer, if the subword in tokenizer_t3.vocab doesn't show up in the instance, the TfIdVectorizer will drop those never appeared. Thus, the number of columns became smaller than the vocab_size. We changed the code and consolidate the columns size with the vocab_size using the index of the vocab

In [62]:
clf.coef_.shape

(1, 800)

In [66]:
clf.coef_[0]

array([ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
        0.00000000e+00,  0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
       -1.01179525e+00, -4.96835318e-01,  1.88470083e-01, -2.82319580e-01,
        0.00000000e+00, -3.71782531e-01, -1.09402337e-01,  4.97677994e-01,
       -2.52731970e-02, -1.46653208e-01,  5.21784110e-02, -3.39885377e-01,
        1.05294600e-01, -2.71235007e-01, -1.69452536e+00,  5.36728867e-01,
        2.02591967e-01,  1.28130954e+00, -4.20858743e-01,  4.59129976e-01,
        2.26522583e-01,  

Shu_solution:
Now we need to find the 5 subwords that contribute the most to the classification of the sentence in Bokmål.
Because we map {"nob":0, "nno":1}, then if the weight is positive, it means it contribute to nynosk, likewise, when the weight is negative, it contribute to bokmål. So now the probelm became to find the top 5 smallest negative weight. 

In [77]:
coef=clf.coef_[0]
top_bokmal_index = np.argsort(coef)[:5]
top_bokmal_index

array([658, 592, 616, 583, 714])

In [78]:
for i in top_bokmal_index:
    print(coef[i]) 

-9.212245299105692
-7.182426336838028
-7.129837130541972
-6.567549063084499
-6.538085758212796


In [80]:
[tokenizer_t3.vocab[i] for i in top_bokmal_index]

['ikke ', 'lig ', 'fra ', 'jeg ', 'sier ']

Shu_explaination:
The bokmål and nynorsk mapping realtion can reveal that these discriminators are actually valid.
ikke - ikkje
jeg - eg
fra - frå
sier - seier